In [1]:
import math
import numpy as np
import cv2
# import yolo_preprocessing
# from yolo_preprocessing import read_annotations

In [2]:
def read_annotations(img_dir, edge_threshold, side_threshold):

    all_imgs = []

    # creating empty box of list of arrays where images would be later stored in
    seen_labels = {}
    # labels would be soon appended in this empty dict list

    for ann_file_name in sorted(os.listdir(img_dir)):

        # creates a list of names in the given image directory (img_dir)
        # sorted() function
        ext = os.path.splitext(ann_file_name)[1]

        # splits the pathname into the its name and its extension (basically ext is index 1 of the total file name eg:filenam.ext))

        if ext == ".txt":
            img = {"object": []}
            # instantisation

            img_file_name = img_dir + ann_file_name

            # technically contains the absolute(abs) path to the file ann_file_name

            # print("img_file_name= ", img_file_name)

            # print( "ann_file_name= ", ann_file_name)

            # read in predefined order
            file = open(img_file_name, "r")

            for line in file:

                # each line is a keypoint pair to this picture
                # file is a list of strings read from the txt file where each line is treated as a string (return or enter at the end of each line is used to determine the end of each string)
                vWords = line.split()

                # splitting the line string into a list of elements, where each element is a sub string of the main string called 'line', using whitespace as the spacer element
                obj = {}

                # an empty dict is created

                img["filename"] = os.path.splitext(img_file_name)[0] + ".bmp"
                # absolute path of the image file for the current annotation file being read, is stored as a string in img['filename']
                # where img is a dictionary

                if (edge_threshold < float(vWords[5])) and (
                    side_threshold < float(vWords[6])
                ):

                    obj["name"] = vWords[0]
                    # in the empty obj dict a class name is created.
                    # this name is appended by index 0 of vWords which is nothing but class

                    # the following if block is for incrementing the number of classes in the empty list of seen_lables={}
                    if obj["name"] in seen_labels:
                        seen_labels[obj["name"]] += 1
                    # now as class (0) is now visible in seen_labels for every iteration 1 is appended
                    else:
                        seen_labels[obj["name"]] = 1
                    # initally if obj['name'] is not in seen_labels then its appended by 1

                    # two keypoints which give pose (x0, y0) and direction (x1, y1) in pixel coordinates, all following coordinates are from object polygon and thus ignored
                    obj["x0"] = float(vWords[1])
                    # x0
                    obj["y0"] = float(vWords[2])
                    # y0

                    angle_alpha = float(vWords[3])

                    alpha_deg = np.rad2deg(angle_alpha)
                    slope=math.tan(alpha_deg)
                    length=10
                    obj["x1"] = obj["x0"]+math.sqrt(length**2/(1+slope**2))
                    obj["y1"] = (slope*(obj["x1"]-obj["x0"]))+obj["y0"]

                    img["object"]+=[obj]

                    # the list called object is in itself stored as a key-value pair within a dictionary called img. the dict img contains two elements {'object', 'filename(abs path to img file)'}
                else:
                    print("Shaft was not considered for training \n")
            file.close()

            if len(img["object"]) > 0:
                # if the object dict is no longer empty then store all the string of data stored in img, in list of all_imgs
                all_imgs += [img]
    return all_imgs, seen_labels

In [4]:
read_annotations("D:\\Master Project\\github\\kk\\Masters-Project\\data\\shafts\\likith\\Shafts_radians\\train\\",0.5,0.5)


Shaft was not considered for training 

Shaft was not considered for training 

Shaft was not considered for training 

Shaft was not considered for training 



02806,
     'y1': 194.95512642281503},
    {'name': '0',
     'x0': 17.949569702148438,
     'y0': 124.45912170410156,
     'x1': 18.481764560739677,
     'y1': 134.44495009399935}],
   'filename': 'D:\\Master Project\\github\\kk\\Masters-Project\\data\\shafts\\likith\\Shafts_radians\\train\\Img_000185.bmp'},
  {'object': [{'name': '0',
     'x0': 145.4290008544922,
     'y0': 155.97429656982422,
     'x1': 145.48754750406476,
     'y1': 145.9744679568021},
    {'name': '0',
     'x0': 50.14259719848633,
     'y0': 178.5709228515625,
     'x1': 55.544609292565106,
     'y1': 186.98628174678655},
    {'name': '0',
     'x0': 105.21833801269531,
     'y0': 135.90018463134766,
     'x1': 114.43601325253364,
     'y1': 132.02275466126804},
    {'name': '0',
     'x0': 182.95762634277344,
     'y0': 115.76449584960938,
     'x1': 189.36008075661965,
     'y1': 123.44619992348653},
    {'name': '0',
     'x0': 93.4107666015625,
     'y0': 185.19448852539062,
     'x1': 103.3653158364008,
   

In [5]:
def draw_kpp(image, g_wdt, g_hgt, kpps, labels, cls_threshold=0.8):
    image_h, image_w, _ = image.shape

    for kpp in kpps:

        x0 = int(kpp.x0)
        y0 = int(kpp.y0)

        alpha = (kpp.alpha_norm - 0.1) / 0.8 * math.pi
        x1 = int(kpp.x0 + math.cos(alpha) * 20)
        y1 = int(kpp.y0 + math.sin(alpha) * 20)

        cv2.circle(image, (x0, y0), 4, (0, 0, 255), 1)
        cv2.line(image, (x0, y0), (x1, y1), (0, 0, 255), 1)

    return image


In [ ]:
cv2.circle(image,(x0,y0),4,(0,255,0),1)
cv2.line(image,(x0,y0),(x1,y1),(0,255,0),1)